Data sources:
- http://crtm.maps.arcgis.com/home/item.html?id=1a25440bf66f499bae2657ec7fb40144
- https://data.renfe.com/dataset/volumen-de-viajeros-por-franja-horaria-madrid

In [1]:
import datetime
import pandas as pd
import os
from statistics import mean
from decimal import Decimal as D

In [2]:
DATA_PATH = '../Data/madrid/'
PARSED_PATH = DATA_PATH + 'parsed/'

if not os.path.exists(PARSED_PATH):
    os.mkdir(PARSED_PATH)

In [3]:
# Due to missing data in the timetables, skip some stations and lines
skip_stations = {
    'COLMENAR VIEJO': [],
    'TRES CANTOS': [],
    'GOLOSO EL': [],
    'SANTA MARIA DE LA ALAMEDA': [],
    'ROBLEDO DE CHAVELA': [],
    'ZARZALEJO': [],
    'ESCORIAL EL': [],
    'ZORRERAS LAS': [],
    'SAN YAGO': [],
    'VILLALBA': ['C3'],
    'GALAPAGAR-LA NAVATA': ['C3'],
    'TORRELODONES': ['C3'],
    'MATAS LAS': ['C3'],
    'PINAR DE LAS ROZAS': ['C3'],
    'PITIS': ['C3'],
    'MIRASIERRA-PACO DE LUCIA': [],
    'RAMON Y CAJAL': ['C3']
}
skip_lines = ['C9']

In [4]:
stop_times = pd.read_csv(DATA_PATH + 'trains/stop_times.txt', index_col=False)
stops = pd.read_csv(DATA_PATH + 'trains/stops.txt', index_col=False)
trips = pd.read_csv(DATA_PATH + 'trains/trips.txt', index_col=False)
routes = pd.read_csv(DATA_PATH + 'trains/routes.txt', index_col=False)
turnstiles = pd.read_csv(DATA_PATH + 'turnstiles.csv', index_col=False)

In [5]:
route2line = {}
for i, row in routes.iterrows():
    route2line[row['route_id']] = row['route_short_name']

stop2name = {}
for i, row in stops.iterrows():
    stop2name[row['stop_id']] = row['stop_name']

trip2route = {}
for i, row in trips.iterrows():
    trip2route[row['trip_id']] = row['route_id']

In [6]:
# List of stations of each line
stations_in_line = {}
lines_in_station = {}

for i, row in stop_times.iterrows():
    line_name = route2line[trip2route[row['trip_id']]]
    from_station = stop2name[row['stop_id']]
    
    if line_name in skip_lines:
        continue
    elif from_station in skip_stations and (line_name in skip_stations[from_station] or len(skip_stations[from_station]) == 0):
        continue
    
    if line_name not in stations_in_line:
        stations_in_line[line_name] = set()
    if from_station not in lines_in_station:
        lines_in_station[from_station] = set()
    
    stations_in_line[line_name].add(from_station)
    lines_in_station[from_station].add(line_name)

## Fix turnstile initial data

In [7]:
# Check that all stations in connections are in the turnstiles table
remove_rows = []
for i, row in turnstiles.iterrows():
    st_name = row['STATION_NAME']
    if st_name not in skip_stations:
        continue
    
    skip_in_lines = skip_stations[st_name]
    
    if len(skip_in_lines) == 0:
        remove_rows.append(i)


for st_name, skip_in_lines in skip_stations.items():
    if st_name not in lines_in_station:
        continue
    
    lines = lines_in_station[st_name]
    
    if len(skip_in_lines) == 0:
        del lines_in_station[st_name]
        stations_in_line = {l:[s for s in stations if s != st_name] for l, stations in stations_in_line.items()}
    else:
        skip_in_lines = [l for l in skip_in_lines if l in lines]
        if skip_in_lines == 0:
            # Consider as already deleted
            continue
        
        lines_in_station[st_name] = [l for l in lines_in_station[st_name] if l not in skip_in_lines]
        stations_in_line = {l:[s for s in stations if s != st_name and l == skip_in_lines] for l, stations in stations_in_line.items()}

        if len(lines_in_station[st_name]) == 0:
            del lines_in_station[st_name]

turnstiles = turnstiles.drop(remove_rows)
turnstiles = turnstiles.reset_index(drop=True)

In [8]:
turnstiles.to_csv(PARSED_PATH + 'turnstiles.csv', index=False)

In [9]:
turnstiles = pd.read_csv(PARSED_PATH + 'turnstiles.csv', index_col=False)

## Connected stations

Note that trains may skip some stations (fast routes) so, in such cases, I consider that a station is connected to the next non-skipped station too.

The final CSV should be: `station_connections.csv`
- `LINE`
- `FROM_STATION`
- `TO_STATION`
- `DIRECTION`

In [10]:
connections = {
    'LINE': [],
    'FROM_STATION': [],
    'TO_STATION': [],
    'DIRECTION': []
}
connections = pd.DataFrame(connections, columns=connections.keys()).astype({'DIRECTION': 'int32'})

for i, row in stop_times.iterrows():
    stop_sequence = row['stop_sequence']
    next_station = stop_times.loc[(stop_times['trip_id'] == row['trip_id']) & (stop_times['stop_sequence'] == stop_sequence + 1)]
    
    if next_station.size == 0:
        continue
    
    direction = trips.loc[trips['trip_id'] == row['trip_id']].iloc[0]['direction_id']
    direction = int(direction)
    
    next_station = next_station.iloc[0]
    
    line_name = route2line[trip2route[row['trip_id']]]
    from_station = stop2name[row['stop_id']]
    to_station = stop2name[next_station['stop_id']]
    
    if line_name in skip_lines:
        continue

    s_row = pd.Series({'LINE': line_name, 'FROM_STATION': from_station, 'TO_STATION': to_station, 'DIRECTION': direction})
    connections = connections.append(s_row, ignore_index=True)
    
connections.drop_duplicates(keep='first', inplace=True)

In [11]:
connections.to_csv(PARSED_PATH + 'station_connections.csv', index=False)

In [12]:
connections = pd.read_csv(PARSED_PATH + 'station_connections.csv', index_col=False)

In [13]:
# Remove stations
found_rows = True
while found_rows:
    to_remove = []
    to_append = []
    found_rows = False
    
    for i, row in connections.iterrows():
        st_name = row['TO_STATION']
        line_name = row['LINE']
        direction = row['DIRECTION']

        if st_name not in skip_stations:
            continue

        skip_st_lines = skip_stations[st_name]
        if len(skip_st_lines) > 0:
            if line_name not in skip_st_lines:
                continue
            else:
                conditions = (connections['LINE'] == line_name) & (connections['DIRECTION'] == direction)
        else:
            conditions = (connections['DIRECTION'] == direction)


        st_skip = connections.loc[(connections['FROM_STATION'] == st_name) & conditions]

        prev_stations = []
        next_stations = st_skip['TO_STATION'].tolist()
        to_remove = to_remove + list(st_skip.index)
        
        for st_skip_i, st_skip_row in st_skip.iterrows():
            st_prev = connections.loc[(connections['TO_STATION'] == st_name) & conditions]
            for _, st_prev_row in st_prev.iterrows():
                if len(next_station) > 0:
                    prev_stations.append(st_prev_row['FROM_STATION'])
            to_remove = to_remove + list(st_prev.index)

        for prev_station in prev_stations:
            for next_station in next_stations:
                s_row = pd.Series({'LINE': line_name, 'FROM_STATION': prev_station,
                                   'TO_STATION': next_station, 'DIRECTION': direction})
                to_append.append(s_row)
        
        
    connections = connections.drop(to_remove)
    for row in to_append:
        connections = connections.append(row, ignore_index=True)

    connections = connections.reset_index(drop=True)
    connections.drop_duplicates(keep='first', inplace=True)
    
    found_rows = len(to_remove) > 0 or len(to_append) > 0

In [14]:
connections.to_csv(PARSED_PATH + 'station_connections.csv', index=False)

In [15]:
connections = pd.read_csv(PARSED_PATH + 'station_connections.csv', index_col=False)

In [16]:
# Check that all stations in connections are in the turnstiles table

for i, row in turnstiles.iterrows():
    st_name = row['STATION_NAME']
    
    exists = connections.loc[connections['FROM_STATION'] == st_name]
    if exists.size == 0:
        raise Exception('Unknown station: %s' % st_name)

print('Ok!')

Ok!


## Timetable

The final file `timetable.csv` should be as follows:
- `ID`: unique ID which represents the train or the vehicle.
- `LINE`: Name of the line.
- `ARRIVAL_TIME`: None or datetime.time (None if it is the first station).
- `DEPARTURE_TIME`: None or datetime.time (None if it is the last station).
- `STATION_NAME`: Station name.
- `NEXT_STATION`: Name of the next station.
- `IS_START`: 1 if it is the starting station of the route.
- `IS_END`: 1 if it is the ending station of the route.
- `DIRECTION`: 1 or 0.

In [17]:
timetable = {
    'ID': [],
    'LINE': [],
    'ARRIVAL_TIME': [],
    'DEPARTURE_TIME': [],
    'STATION_NAME': [],
    'NEXT_STATION': [],
    'IS_START': [],
    'IS_END': [],
    'DIRECTION': []
}
timetable = pd.DataFrame(timetable, columns=timetable.keys()).astype({'ID': 'int32', 'DIRECTION': 'int32',
                                                                      'IS_START': 'int32', 'IS_END': 'int32'})

trip_ids = stop_times['trip_id'].unique()
trip2train = {trip_id:i for i, trip_id in enumerate(trip_ids)}

# We must fix the timezone of the data
time_offset = 1

for i, row in stop_times.iterrows():
    stop_sequence = row['stop_sequence']
    next_station = stop_times.loc[(stop_times['trip_id'] == row['trip_id']) & (stop_times['stop_sequence'] == stop_sequence + 1)]
    
    train_id = int(trip2train[row['trip_id']])
    line_name = route2line[trip2route[row['trip_id']]]
    station_name = stop2name[row['stop_id']]
    
    direction = trips.loc[trips['trip_id'] == row['trip_id']].iloc[0]['direction_id']
    direction = int(direction)
    
    if line_name in skip_lines:
        continue
    
    if next_station.size > 0:
        next_station = next_station.iloc[0]
        next_station = stop2name[next_station['stop_id']]
    else:
        next_station = None
    
    arrival_time = row['arrival_time'].split(':')
    arrival_hour = int(arrival_time[0]) + time_offset if int(arrival_time[0]) + time_offset < 24 else 0
    arrival_time = datetime.time(arrival_hour, int(arrival_time[1]))
    
    departure_time = row['departure_time'].split(':')
    departure_hour = int(departure_time[0]) + time_offset if int(departure_time[0]) + time_offset < 24 else 0
    departure_time = datetime.time(departure_hour, int(departure_time[1]))
    
    is_start = int(stop_sequence) == 0
    is_end = next_station is None

    s_row = pd.Series({'ID': train_id, 'LINE': line_name, 'ARRIVAL_TIME': arrival_time,
                       'DEPARTURE_TIME': departure_time, 'STATION_NAME': station_name,
                       'NEXT_STATION': next_station, 'IS_START': is_start, 'IS_END': is_end,
                       'DIRECTION': direction})
    timetable = timetable.append(s_row, ignore_index=True)

In [18]:
# Remove stations
found_rows = True
while found_rows:
    found_rows = False
    to_remove = []
    for skip_station_name, skip_lines in skip_stations.items():
        if len(skip_lines) == 0:
            st_timetable = timetable.loc[(timetable['STATION_NAME'] == skip_station_name)]
        else:
            st_timetable = timetable.loc[(timetable['STATION_NAME'] == skip_station_name) &
                                         (timetable['LINE'].isin(skip_lines))]

        for i, row in st_timetable.iterrows():
            found_rows = True
            prev_timetable = timetable.loc[(timetable['NEXT_STATION'] == row['STATION_NAME']) &
                                           (timetable['ID'] == row['ID'])]
            has_previous_station = (prev_timetable.size > 0)

            next_timetable = timetable.loc[(timetable['STATION_NAME'] == row['NEXT_STATION']) &
                                           (timetable['ID'] == row['ID'])]
            has_next_station = (next_timetable.size > 0)
            
            for next_i, next_row in next_timetable.iterrows():
                if not has_previous_station:
                    timetable.at[prev_i, 'IS_START'] = 1
            
            for prev_i, prev_row in prev_timetable.iterrows():
                if not has_next_station:
                    timetable.at[prev_i, 'DEPARTURE_TIME'] = row['ARRIVAL_TIME']
                    timetable.at[prev_i, 'NEXT_STATION'] = None
                    timetable.at[prev_i, 'IS_END'] = 1
                else:
                    timetable.at[prev_i, 'NEXT_STATION'] = next_timetable.iloc[0]['STATION_NAME']
                
            to_remove.append(i)

    timetable = timetable.drop(to_remove)
    timetable = timetable.reset_index(drop=True)

In [19]:
timetable.to_csv(PARSED_PATH + 'timetable.csv', index=False)

In [20]:
timetable = pd.read_csv(PARSED_PATH + 'timetable.csv', index_col=False)

## Weights of lines

Based on the average number of IN-passengers of the stations of the line, compute the weight of the line in the network.

In [21]:
w_lines = {
    'LINE': [],
    'WEIGHT_IN': [],
    'WEIGHT_OUT': []
}
w_lines = pd.DataFrame(w_lines, columns=w_lines.keys())

all_line_inweights = {}
all_line_outweights = {}

for i, row in turnstiles.iterrows():
    # Get name of station and all the lines in the station
    st_name = row['STATION_NAME']
    line_names = lines_in_station[st_name]
    
    in_count = D(row['IN']) / len(line_names)
    out_count = D(row['OUT']) / len(line_names)
    
    for line_name in line_names:
        if line_name not in all_line_inweights:
            all_line_inweights[line_name] = []
            all_line_outweights[line_name] = []
        
        all_line_inweights[line_name].append(in_count)
        all_line_outweights[line_name].append(out_count)

# Calculate means
all_line_inweights = {line_name:mean(w) for line_name, w in all_line_inweights.items()}
all_line_outweights = {line_name:mean(w) for line_name, w in all_line_outweights.items()}

# Normalize and store
norm_in = D(sum(list(all_line_inweights.values())))
norm_out = D(sum(list(all_line_outweights.values())))

line_names = timetable['LINE'].unique()
for line_name in line_names:
    weight_in = all_line_inweights[line_name]
    weight_out = all_line_outweights[line_name]
    
    weight_in = D(weight_in) / norm_in
    weight_out = D(weight_out) / norm_out
    
    s_row = pd.Series({'LINE': line_name, 'WEIGHT_IN': weight_in, 'WEIGHT_OUT': weight_out})
    w_lines = w_lines.append(s_row, ignore_index=True)

In [22]:
w_lines.to_csv(PARSED_PATH + 'line_weights.csv', index=False)

In [23]:
w_lines = pd.read_csv(PARSED_PATH + 'line_weights.csv', index_col=False)

## Weights of stations in the line

Depending on the time, each station of the line has a specific weight.

**TO DO:** It should be calculated using some kind of continuous probability function.

**Note**: The sum of weights of all stations (in the same time) on each line must be 1.

In [24]:
dict_w_lines = w_lines.set_index(['LINE']).to_dict()

In [26]:
w_stations = {
    'LINE': [],
    'STATION_NAME': [],
    'TIME': [],
    'WEIGHT_IN': [],
    'WEIGHT_OUT': []
}
w_stations = pd.DataFrame(w_stations, columns=w_stations.keys()).set_index(['LINE', 'STATION_NAME', 'TIME'])

all_line_weights = {}

debug_prev_station = None
debug_i = 0
debug_len = len(turnstiles['STATION_NAME'].unique())
for i, row in turnstiles.iterrows():
    if row['STATION_NAME'] != debug_prev_station:
        debug_prev_station = row['STATION_NAME']
        debug_i = debug_i + 1
        print('\r%s (%d of %d)'% (row['STATION_NAME'], debug_i, debug_len), ' '*20, end='')
    
    # Get the length of the time interval in the turnstile table
    time_range = [t.strip().split(':') for t in turnstiles.at[i, 'TIME_INTERVAL'].split('-')]
    delta_start = datetime.timedelta(hours=int(time_range[0][0]), minutes=int(time_range[0][1]))
    delta_end = datetime.timedelta(hours=int(time_range[1][0]), minutes=int(time_range[1][1]))
    
    if delta_end < delta_start:
        delta_mid = datetime.timedelta(hours=23, minutes=59)
        total_seconds = (delta_mid - delta_start).total_seconds()
        
        total_seconds = total_seconds + delta_end.total_seconds()
        
        n_minutes = int((total_seconds) // 60) + 1
    else:
        total_seconds = (delta_end - delta_start).total_seconds()
        n_minutes = int(total_seconds // 60)
    
    # Get name of station and all the lines in the station
    st_name = row['STATION_NAME']
    line_names = lines_in_station[st_name]
    
    in_count = D(row['IN']) / n_minutes
    out_count = D(row['OUT']) / n_minutes
    
    lines_station_weight_in = w_lines.loc[w_lines['LINE'].isin(line_names)]['WEIGHT_IN'].sum()
    lines_station_weight_out = w_lines.loc[w_lines['LINE'].isin(line_names)]['WEIGHT_OUT'].sum()
    time_ref = datetime.datetime(100, 1, 1, int(time_range[0][0]), int(time_range[0][1]), 0)
    
    for line_name in line_names:
        # If the station is shared with more lines, get the weight of the line
        line_weight_in = dict_w_lines['WEIGHT_IN'][line_name]
        line_weight_out = dict_w_lines['WEIGHT_OUT'][line_name]
        w_in = D(line_weight_in) / D(lines_station_weight_in)
        w_out = D(line_weight_out) / D(lines_station_weight_out)
        
        # Add one row per minute
        for k in range(n_minutes):
            k_time = (time_ref + datetime.timedelta(minutes=k)).time()
            
            s_row = pd.Series({'WEIGHT_IN': in_count * w_in,
                               'WEIGHT_OUT': out_count * w_out}, name=(line_name, st_name, k_time))
            w_stations = w_stations.append(s_row)

print('\nDone!')

AEROPUERTO T4 (83 of 83)                                     
Done!


In [27]:
w_stations.to_csv(PARSED_PATH + 'station_weights.csv')

In [28]:
# Normalize data
w_stations = pd.read_csv(PARSED_PATH + 'station_weights.csv', index_col=['LINE', 'STATION_NAME', 'TIME'])

w_normstations = w_stations.copy()
time_groups = w_stations.groupby(['LINE', 'TIME']).sum()

for i, row in w_normstations.iterrows():
    line_name = i[0]
    time = i[2]
    norm_in = D(time_groups.at[(line_name, time), 'WEIGHT_IN'])
    norm_out = D(time_groups.at[(line_name, time), 'WEIGHT_OUT'])
    w_normstations.at[i, 'WEIGHT_IN'] = D(0) if norm_in == 0 else D(w_normstations.at[i, 'WEIGHT_IN']) / norm_in
    w_normstations.at[i, 'WEIGHT_OUT'] = D(0) if norm_out == 0 else D(w_normstations.at[i, 'WEIGHT_OUT']) / norm_out

w_normstations.head()

WEIGHT_IN  WEIGHT_OUT
LINE STATION_NAME TIME                           
C10  PRINCIPE PIO 00:00:00        0.0    0.104674
                  00:01:00        0.0    0.104674
                  00:02:00        0.0    0.104674
                  00:03:00        0.0    0.104674
                  00:04:00        0.0    0.104674

In [29]:
w_normstations.to_csv(PARSED_PATH + 'norm_station_weights.csv')

In [30]:
w_normstations = pd.read_csv(PARSED_PATH + 'norm_station_weights.csv')

## Time weights

In [31]:
w_times = {
    'TIME': [],
    'WEIGHT_IN': [],
    'WEIGHT_OUT': []
}
w_times = pd.DataFrame(w_times, columns=w_times.keys()).set_index('TIME')

for i, row in turnstiles.iterrows():
    # Get the length of the time interval in the turnstile table
    time_range = [t.strip().split(':') for t in turnstiles.at[i, 'TIME_INTERVAL'].split('-')]
    delta_start = datetime.timedelta(hours=int(time_range[0][0]), minutes=int(time_range[0][1]))
    delta_end = datetime.timedelta(hours=int(time_range[1][0]), minutes=int(time_range[1][1]))
    
    if delta_end < delta_start:
        delta_mid = datetime.timedelta(hours=23, minutes=59)
        total_seconds = (delta_mid - delta_start).total_seconds()
        
        total_seconds = total_seconds + delta_end.total_seconds()
        
        n_minutes = int((total_seconds) // 60) + 1
    else:
        total_seconds = (delta_end - delta_start).total_seconds()
        n_minutes = int(total_seconds // 60)
    
    in_count = D(row['IN']) / n_minutes
    out_count = D(row['OUT']) / n_minutes
    
    # Add one row per minute
    time_start = datetime.datetime(100, 1, 1, int(time_range[0][0]), int(time_range[0][1]), 0)
    for k in range(n_minutes):
        k_time = (time_start + datetime.timedelta(minutes=k)).time()
        
        if k_time not in w_times.index:
            w_times = w_times.append(pd.Series({'WEIGHT_IN': D(0), 'WEIGHT_OUT': D(0)}, name=k_time))

        w_times.at[k_time, 'WEIGHT_IN'] = w_times.at[k_time, 'WEIGHT_IN'] + in_count
        w_times.at[k_time, 'WEIGHT_OUT'] = w_times.at[k_time, 'WEIGHT_OUT'] + out_count

w_times['WEIGHT_IN'] = w_times['WEIGHT_IN'] / turnstiles['IN'].sum()
w_times['WEIGHT_OUT'] = w_times['WEIGHT_OUT'] / turnstiles['OUT'].sum()

In [32]:
w_times.to_csv(PARSED_PATH + 'times_weights.csv')

## Coordinates of stations

In [33]:
stations_coords = {
    'STATION_NAME': [],
    'X': [],
    'Y': []
}
stations_coords = pd.DataFrame(stations_coords, columns=stations_coords.keys()).set_index('STATION_NAME')

station_names = connections.reset_index()['FROM_STATION'].unique()

for i, row in stops.iterrows():
    if row['stop_name'] not in station_names:
        continue
    
    stations_coords = stations_coords.append(pd.Series({'X': row['stop_lon'],
                                                        'Y': row['stop_lat']}, name=row['stop_name']))

In [34]:
stations_coords.to_csv(PARSED_PATH + 'station_coords.csv')